In [1]:
import timeit
import theano
from theano import tensor as T
import numpy as np
from datetime import datetime

from ml_time_series_theano import generate_envelope_T
from ml_time_series_theano import generate_envelope_nd_T

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce 940MX (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5110)


# Python function

In [2]:
def generate_envelope(original_data, num_samples):
    new_Xc = np.array([])
    for i in np.arange(num_samples - 1, original_data.shape[0]):
        #x = Xc_1[i - (NUM_SAMPLES - 1) : i+1, :].ravel()
        x = original_data[i - (num_samples - 1) : i+1].ravel()        
        if new_Xc.shape[0] == 0:
            new_Xc = x
        else:
            new_Xc = np.vstack((new_Xc, x))
    return new_Xc

def proccess_data(file_,DATA):
    
    #datafile = os.path.join(DATADIR, DATAFILE)
    #datafile = DATAFILE

    # Remove header, Nan and trash
    data = np.load('DB_GRIP/'+file_+'.npy')
    Xc_1 = data[:10000 ,0]
    Xc_2 = data[10000: ,0]

    
    print '\nXc_1 cleaned shape ', Xc_1.shape
    #print 'Xc_2 cleaned shape ', Xc_2.shape
    
    #Create temporal serie
    NUM_SAMPLES = 50
    #NUM_COLS = Xc_1.shape[1]

    
    Xc_1 = generate_envelope(Xc_1, NUM_SAMPLES)
    Xc_2 = generate_envelope(Xc_2, NUM_SAMPLES)
    
    
    print 'Xc_1 temporal-serie shape ', Xc_1.shape
    print 'Xc_2 temporal-serie shape ', Xc_2.shape

# Theano function

In [3]:

def generate_envelope_T(original_data, num_samples):
    # initialize
    new_Xc = np.array([])
    x = T.fmatrix('x')
    w = theano.shared(num_samples-1)
    update = [(w, w + 1)]
    envelope1d = x[0][w - (num_samples - 1) : w+1].ravel()

    # compile model
    net_input = theano.function(inputs=[x],
                                updates=update,
                                outputs=envelope1d)

    # execute
    X_input =  np.array([(original_data)], 
                          dtype=theano.config.floatX)
    if num_samples < len(original_data):
        new_Xc = net_input(X_input) 
        for i in np.arange(num_samples , original_data.shape[0]):
            new_Xc = np.vstack((new_Xc, net_input(X_input)))
    else:
        print 'num_samples must be lower than original_data.'
        
    return new_Xc

def generate_envelope_nd_T(original_data, num_samples):
    # initialize
    new_Xc = np.array([])
    x = T.dmatrix('x')
    w = theano.shared(num_samples-1)
    update = [(w, w + 1)]
    envelope_nd = x[w-(num_samples-1):w+1][0 : num_samples, :].ravel()
    
    # compile model
    net_input = theano.function(inputs=[x],
                                updates=update,
                                outputs=envelope_nd)
    # execute
    X_input =  np.array((original_data), 
                          dtype=theano.config.floatX)

    if num_samples < len(original_data):
        new_Xc = net_input(X_input) 
        for i in np.arange(num_samples, original_data.shape[0]):
            new_Xc = np.vstack((new_Xc, net_input(X_input)))

    else:
        print 'num_samples must be lower than original_data.'
    return new_Xc

In [415]:
num_samples = 4

X_input =  np.array(np.ndarray((10,4), 
                                buffer=np.array(np.arange(0,40)),
                                dtype=int),dtype=theano.config.floatX)


x = T.dmatrix('x')

w = theano.shared(num_samples-1)

update = [(w, w + 1)]

calc_series = x[w-(num_samples-1):w+1][0 : num_samples, :].ravel()


# compile model
net_input = theano.function(inputs=[x],
                            updates=update,
                            outputs=calc_series)

# execute
new_Xc = np.array([])
new_Xc = net_input(X_input) 

for i in np.arange(num_samples, original_data.shape[0]):
    new_Xc = np.vstack((new_Xc, net_input(X_input)))


print new_Xc.shape
print new_Xc




(7, 16)
[[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
   14.  15.]
 [  4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.  17.
   18.  19.]
 [  8.   9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  21.
   22.  23.]
 [ 12.  13.  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.
   26.  27.]
 [ 16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
   30.  31.]
 [ 20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.  31.  32.  33.
   34.  35.]
 [ 24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.
   38.  39.]]


In [378]:
np.array(np.arange(0,40))
JJ = np.ndarray((10,4), buffer=np.array(np.arange(0,40)),dtype=int)
#print JJ[2:6][0:4,:]
w = 9
num_samples = 4
for i in range(len(np.arange(num_samples - 1, original_data.shape[0]))):
    w = 3+i
    print (JJ[w-(num_samples-1):w+1][0 : num_samples, :].ravel()
)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
[20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35]
[24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]


In [473]:
num_samples = 4
original_data = (np.ndarray((10,4), buffer=np.array(np.arange(0,40)),dtype=int))
#original_data = np.array(np.array(np.arange(0,20)))
#proccess_data_T(original_data,DATA)
generate_envelope_nd_T(original_data,4)
#generate_envelope_T(original_data,4)

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.],
       [  4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,
         15.,  16.,  17.,  18.,  19.],
       [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,
         19.,  20.,  21.,  22.,  23.],
       [ 12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
         23.,  24.,  25.,  26.,  27.],
       [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,
         27.,  28.,  29.,  30.,  31.],
       [ 20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,
         31.,  32.,  33.,  34.,  35.],
       [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,
         35.,  36.,  37.,  38.,  39.]])

In [447]:
proccess_data(original_data,DATA)
generate_envelope(original_data,4)


Xc_1 cleaned shape  (10, 4)
Xc_1 temporal-serie shape  (7, 16)


array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
       [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
       [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
       [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]])

In [4]:
def proccess_data_T(file_,DATA):
    
    #datafile = os.path.join(DATADIR, DATAFILE)
    #datafile = DATAFILE

    # Remove header, Nan and trash
    data = np.load('DB_GRIP/'+file_+'.npy')
    Xc_1 = data[:10000 ,0]
    Xc_2 = data[10000: ,0]
    
    print '\nXc_1 cleaned shape ', Xc_1.shape
    print 'Xc_2 cleaned shape ', Xc_2.shape
    
    #Create temporal serie
    NUM_SAMPLES = 50
    #NUM_COLS = Xc_1.shape[1]

    
    Xc_1 = generate_envelope_T(Xc_1, NUM_SAMPLES)
    Xc_2 = generate_envelope_T(Xc_2, NUM_SAMPLES)
    
    
    print 'Xc_1 temporal-serie shape ', Xc_1.shape
    print 'Xc_2 temporal-serie shape ', Xc_2.shape
    

## Without Theano

In [9]:
start = timeit.default_timer()

files = ['data_220833','data_220853','data_220915','data_220933', 'data_220951']
DATA = []
for i in files:
    HH = proccess_data(i,DATA)
    
stop = timeit.default_timer()
ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))


Xc_1 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)

Xc_1 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)

Xc_1 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)

Xc_1 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)

Xc_1 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)
Tempo total para o envelopamento: 0.52 min


## With Theano

In [10]:
start = timeit.default_timer()

files = ['data_220833','data_220853','data_220915','data_220933', 'data_220951']
DATA = []
for i in files:
    proccess_data_T(i,DATA)
    
stop = timeit.default_timer()
ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))


Xc_1 cleaned shape  (10000,)
Xc_2 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)

Xc_1 cleaned shape  (10000,)
Xc_2 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)

Xc_1 cleaned shape  (10000,)
Xc_2 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)

Xc_1 cleaned shape  (10000,)
Xc_2 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)

Xc_1 cleaned shape  (10000,)
Xc_2 cleaned shape  (10000,)
Xc_1 temporal-serie shape  (9951, 50)
Xc_2 temporal-serie shape  (9951, 50)
Tempo total para o envelopamento: 0.19 min


In [36]:
w = theano.shared(0)

In [296]:
# initialize
x = T.fmatrix('x')
#w = theano.shared(np.asarray([[0.0, 0.0, 0.0]],dtype=theano.config.floatX))
w = theano.shared(0)
#z = x.dot(w.T)
z = x*(w)
update = [(w, w + 1)]


# compile
net_input = theano.function(inputs=[x],
                            updates=update,
                            outputs=z)

# execute
#data = np.array([[1, 2, 3]],dtype=theano.config.floatX)

data = np.array([(np.array(np.arange(0,4)))],dtype=theano.config.floatX)

for i in range(5):
    print('z%d:' % i, net_input(data))


('z0:', array([[ 0.,  0.,  0.,  0.]]))
('z1:', array([[ 0.,  1.,  2.,  3.]]))
('z2:', array([[ 0.,  2.,  4.,  6.]]))
('z3:', array([[ 0.,  3.,  6.,  9.]]))
('z4:', array([[  0.,   4.,   8.,  12.]]))
